In [3]:
import numpy as np
import pandas as pd

df = pd.read_csv('TextFiles/moviereviews.tsv', sep='\t')
df.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


In [6]:
df.isnull().sum()

label      0
review    35
dtype: int64

In [7]:
df.dropna(inplace=True)

In [8]:
blank=[]
for i,lb,rv in df.itertuples():
    if type(rv)==str:
        if rv.isspace():
            blank.append(i)
        

In [9]:
print(blank)

[57, 71, 147, 151, 283, 307, 313, 323, 343, 351, 427, 501, 633, 675, 815, 851, 977, 1079, 1299, 1455, 1493, 1525, 1531, 1763, 1851, 1905, 1993]


In [10]:
df.drop(blank,inplace=True)

In [11]:
len(df)

1938

In [12]:
df['label'].value_counts()

neg    969
pos    969
Name: label, dtype: int64

In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemm = WordNetLemmatizer()
import string

In [36]:
count=0
def text_process(rev):
    global count
    count+=1
    print(count,end="\r")
    no_punc = [word for word in rev if word not in string.punctuation]
    no_punc = "".join(no_punc)
    return " ".join(lemm.lemmatize(word) for word in no_punc.split() if word.lower() not in stopwords.words('english'))

In [37]:
df['clean_review'] = df['review'].apply(text_process)

In [38]:
df

,label,review,clean_review
0,neg,how do films like mouse hunt get into theatres...,film like mouse hunt get theatre isnt law some...
1,neg,some talented actresses are blessed with a dem...,talented actress blessed demonstrated wide act...
2,pos,this has been an extraordinary year for austra...,extraordinary year australian film shine scoop...
3,pos,according to hollywood movies made in last few...,according hollywood movie made last decade lif...
4,neg,my first press screening of 1998 and already i...,first press screening 1998 already ive gotten ...
...,...,...,...
1995,pos,"i like movies with albert brooks , and i reall...",like movie albert brook really like movie dire...
1996,pos,it might surprise some to know that joel and e...,might surprise know joel ethan coen brought un...
1997,pos,the verdict : spine-chilling drama from horror...,verdict spinechilling drama horror maestro ste...
1998,pos,i want to correct what i wrote in a former ret...,want correct wrote former retrospective david ...


In [39]:
from sklearn.model_selection import train_test_split

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [41]:
tf =TfidfVectorizer(ngram_range=[1,1],
                             tokenizer=lambda x: x.split(),
                             binary=True)


In [42]:
X=tf.fit_transform(df['clean_review'])

In [43]:
X_train, X_test,y_train,y_test = train_test_split(X,df['label'],test_size=.3)

# Naive Bayes

In [44]:
from sklearn.naive_bayes import MultinomialNB

In [97]:
mb = MultinomialNB(alpha=.4)

In [98]:
mb.fit(X_train, y_train)

MultinomialNB(alpha=0.4)

In [99]:
mb_prediction = mb.predict(X_test)

In [100]:
from sklearn.metrics import classification_report

In [101]:
print(classification_report(y_test,mb_prediction))

              precision    recall  f1-score   support

         neg       0.86      0.82      0.84       296
         pos       0.82      0.86      0.84       286

    accuracy                           0.84       582
   macro avg       0.84      0.84      0.84       582
weighted avg       0.84      0.84      0.84       582



# SVM

In [102]:
from sklearn.svm import LinearSVC

In [103]:
svc = LinearSVC()

In [104]:
svc.fit(X_train,y_train)

LinearSVC()

In [106]:
svc_prediction = svc.predict(X_test)

In [107]:
print(classification_report(y_test,svc_prediction))

              precision    recall  f1-score   support

         neg       0.87      0.85      0.86       296
         pos       0.85      0.87      0.86       286

    accuracy                           0.86       582
   macro avg       0.86      0.86      0.86       582
weighted avg       0.86      0.86      0.86       582



In [111]:
# YOU DO NOT NEED TO RUN THIS CELL UNLESS YOU HAVE
# RECENTLY OPENED THIS NOTEBOOK OR RESTARTED THE KERNEL:

import numpy as np
import pandas as pd

df = pd.read_csv('TextFiles/moviereviews.tsv', sep='\t')
df.dropna(inplace=True)
blanks = []
for i,lb,rv in df.itertuples():
    if type(rv)==str:
        if rv.isspace():
            blanks.append(i)
df.drop(blanks, inplace=True)
from sklearn.model_selection import train_test_split
X = df['review']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn import metrics

# Naïve Bayes Model:
text_clf_nb = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', MultinomialNB()),
])

# Linear SVC Model:
text_clf_lsvc = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

# Train both models on the moviereviews.tsv training set:
text_clf_nb.fit(X_train, y_train)
text_clf_lsvc.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [112]:
myreview = "A movie I really wanted to love was terrible. \
I'm sure the producers had the best intentions, but the execution was lacking."

In [113]:
print(text_clf_nb.predict([myreview]))  # be sure to put "myreview" inside square brackets

['neg']
